In [ ]:
%cd ..

In [ ]:
import ibkr
from analysis import liveValuesForPositions
from decimal import Decimal
from ib_insync import IB
from model import Cash, Currency, Stock
import pandas as pd

In [ ]:
# TODO: Support multi-currency portfolios
currency = Currency.USD

In [ ]:
desiredAllocation = {
    'TSLA': 0.02,
    'AAPL': 0.05,
    'BRK B': 0.05,
    'VT': 0.8,
    'BND': 0.08,
}

In [ ]:
ib = IB()
ib.connect('127.0.0.1', port = 4001)
positions = ibkr.downloadPositions(ib, lenient=False)

In [ ]:
stockPositions = [p for p in positions if isinstance(p.instrument, Stock) and p.instrument.currency == currency]
stockPositions.sort(key=lambda p: p.instrument)
stockPositions

In [ ]:
values = liveValuesForPositions(stockPositions, ibkr.IBDataProvider(ib))

In [ ]:
portfolioValue = sum((x for x in values.values()), Cash(currency = currency, quantity = Decimal(0)))
str(portfolioValue)

In [ ]:
def color_deviations(val):
    max_deviation = 0.02
    color = 'black'
    if abs(val) > max_deviation:
        if val > 0:
            color = 'green'
        else:
            color = 'red'
            
    return f'color: {color}'

In [ ]:
rows = {p.instrument.symbol: [
    values[p],
    float(values[p].quantity) / float(portfolioValue.quantity),
    desiredAllocation.get(p.instrument.symbol),
    float(values[p].quantity) / float(portfolioValue.quantity) - (desiredAllocation.get(p.instrument.symbol, 0)),
] for p in stockPositions}

In [ ]:
df = pd.DataFrame.from_dict(data=rows, orient='index', columns=[
    'Market value',
    '% of portfolio',
    'Desired %',
    'Deviation'
])

df.style.format({
    '% of portfolio': '{:.2%}',
    'Desired %': '{:.2%}',
    'Deviation': '{:.2%}'
}).applymap(color_deviations, 'Deviation')